In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from fredapi import Fred
#import chart_wizard2 as cw
import matplotlib.pyplot as plt
import openpyxl as ox
import math

In [2]:
### 주요 함수를 정의한다

def get_clean_data(df, year):
    
    con_1 = ~df['시가총액'].isnull()
    con_2 = ~df['자본총계'].isnull()
    con_3 = ~df['P/B(Adj., FY End)'].isnull()
    con_4 = ~df['ROA'].isnull()
    condition_1 = con_1 | con_2 | con_3 | con_4

    ## 우선주자본금 측정시 오류를 제거하기 위한 장치
    df.loc[:, '우선주자본금'] = df['우선주자본금'].fillna(0)

    con_1 = df['P/B(Adj., FY End)'] > 0
    con_2 = df['결산월'] == 12
    con_3 = df['상장일'] < str(year-1)+'-01-01'
    con_4 = df['자본총계'] > df['우선주자본금']

    condition_2 = con_1 & con_2 & con_3 & con_4

    df = df[condition_1 & condition_2]

    #ni=~pd.isnull(df['당기순이익']) 
    #roa=(~pd.isnull(df['ROA']) & ~pd.isnull(df['ROA.1']))
    #ocf = (~pd.isnull(df['영업활동으로인한현금흐름']) & ~pd.isnull(df['영업활동으로인한현금흐름.1']))
    #debt= (~pd.isnull(df['비유동부채']) & ~pd.isnull(df['비유동부채.1']))
    #asset= (~pd.isnull(df['자산총계']) & ~pd.isnull(df['자산총계.1']))
    #turn=(~pd.isnull(df['총자산회전율']) & ~pd.isnull(df['총자산회전율.1']))

    #condition_3= ni & roa & ocf & debt & asset & turn

    #df = df[condition_3]
    ## 수정주가가 누락된 기업들의 정보 제거
    #df = df.dropna(subset=df.filter(like='수정주가').columns)
        ##금융기업 제거
    finance = ['은행', '종금', '화재', '증권', '금융', '보험', '손보', '손해']

    for f in finance:

        df = df[~df['Name'].str.contains(f)]
        
    return df


def get_SMB_HML(fin_df, num1, num2):
    fin_df.loc[:, 'BE_ME'] = fin_df['시가총액']/(fin_df['자본총계'] - fin_df['우선주자본금'])
    fin_df.loc[:, 'SMB_grp'] = pd.qcut(fin_df['시가총액'], num1, labels=range(1,num1+1)) 
    new_series = fin_df.groupby(['SMB_grp'])['BE_ME'].apply(lambda x: pd.qcut(x, num2, labels=range(1, num2+1)))
    new_series.name = 'HML_grp'
    merged_fin_df = pd.concat([fin_df, new_series], axis=1)
    return merged_fin_df


def get_oneperiod_backtest(price_df, company_list, s_date, e_date, initial_money):

    equal_weight = 1/int(len(price_df.columns))
    strategy_price = price_df
    print(strategy_price.head())

    pf_fund_num = {}
    fund_amount = 0
    fund_pf = 0

    each_money_amount = equal_weight*initial_money
    #each_money_list = each_money_array.tolist()

    for i, code in enumerate(company_list):
        print(i, code)
        temp = each_money_amount/(strategy_price.iloc[0][str(code)])
        print(temp)
        temp = math.floor(temp/1)
        temp = temp * 1
        pf_fund_num[code] = temp

        fund_amount = fund_amount + temp*strategy_price[code][0]
        fund_amount = math.floor(fund_amount)
        fund_pf = fund_pf + strategy_price[code]*pf_fund_num[code]
        print(fund_amount)

    cash_amount = initial_money - fund_amount

    backtest_df = pd.DataFrame({'펀드포트폴리오' : fund_pf})
    backtest_df['현금포트폴리오'] = [cash_amount]*len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['펀드포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money -1
    
    return backtest_df

def get_onefactor_return(dict_df, price_data, factor_col, year1, year2, num1, initial_money):

    total_df = 0
    n=0

    for year in range(year1, year2):

        start = str(year)+'-07'
        end = str(year+1)+'-06'
        target_df = dict_df[year]
        temp_df = target_df[target_df[factor_col] == num1] 
        com_list = temp_df.index.values.tolist()
        price_df2 = price_df.loc[start:end][com_list]
        price_df3 = price_df2.dropna(axis=1)

        backtest_df = get_oneperiod_backtest(price_df3, price_df3.columns, start, end, initial_money)

        initial_money=backtest_df['종합포트폴리오'][-1]

        print(year)

        if n == 0:
            total_df = backtest_df
        else:
            total_df = pd.concat([total_df.iloc[:-1], backtest_df])

        n = n+1

        print(total_df)
    total_df['일변화율'] = total_df['종합포트폴리오'].pct_change()
    total_df['총변화율'] = total_df['종합포트폴리오']/total_df['종합포트폴리오'][0]-1
    
    return total_df

def get_return_by_group(dict_df, price_df, col_name1, col_name2, num1, num2, year1, year2, initial_money):

    #initial_money = 100000000
    total_df = 0
    n=0

    for year in range(year1, year2):

        start = str(year)+'-07'
        end = str(year+1)+'-06'
        target_df = company_dict[year]
        temp_df = target_df[(target_df[col_name1] == num1) & (target_df[col_name2] == num2)]
        com_list = temp_df.index.values.tolist()
        price_df2 = price_df.loc[start:end][com_list]
        price_df3 = price_df2.dropna(axis=1)

        backtest_df = get_oneperiod_backtest(price_df3, price_df3.columns, start, end, initial_money)

        initial_money=backtest_df['종합포트폴리오'][-1]

        print(year)

        if n == 0:
            total_df = backtest_df
        else:
            total_df = pd.concat([total_df.iloc[:-1], backtest_df])

        n = n+1

    total_df['일변화율'] = total_df['종합포트폴리오'].pct_change()
    total_df['총변화율'] = total_df['종합포트폴리오']/total_df['종합포트폴리오'][0]-1
    
    return total_df

In [3]:
#print(year) 
path= r'C:\Users\stox1\Desktop\업무\자료수집\factor_model'
path2 = r'C:\Users\stox1\Desktop\research\New_Factor_Model\New_3_factor\전상장기업_코드리스트_since_2008_FF.xlsx'
#year = 2009
#df = pd.read_excel(path+'/'+str(year)+'_F.xlsx', index_col = 'Code')
#df['상장일']=pd.to_datetime(df['상장일'], format='%Y%m%d')
price_df = pd.read_excel(path2, skiprows=[0,1,2,3,4,5,6,8,9,10,11,12,13,14,15], index_col='Code')

In [69]:
company_dict ={}

for year in range(2008, 2020):
    df = pd.read_excel(path+'/'+str(year)+'_F.xlsx', index_col = 'Code')
    df['상장일']=pd.to_datetime(df['상장일'], format='%Y%m%d')
    data_df = get_clean_data(df, year)
    merged_fin_df = get_SMB_HML(data_df, 2, 3)     ## size group수, HML group 수 
    group_df= merged_fin_df[['Name', 'SMB_grp', 'HML_grp']]
    company_dict[year] = group_df

In [71]:
#def get_return_by_group(dict_df, price_df, col_name1, col_name2, num1, num2, year1, year2, initial_money):
total_df = get_return_by_group(company_dict, price_df, 'SMB_grp', 'HML_grp', 1, 1, 2008, 2019, 100000000)

            A000440  A000590  A000760  A000890  A001070  A001140  A001420  \
Code                                                                        
2008-07-01     4435    81635     5290     1762     7720      742     1874   
2008-07-02     4395    78801     5290     1665     7670      738     1796   
2008-07-03     4450    82089     5210     1665     7500      734     1866   
2008-07-04     4105    83109     5120     1627     7600      740     2109   
2008-07-07     4160    83336     5300     1617     7480      746     2001   

            A001770  A001810  A002140  ...  A083550  A083930  A084680  \
Code                                   ...                              
2008-07-01    17150     1975     1080  ...     1826     4650      680   
2008-07-02    16400     1955      918  ...     1708     4109      650   
2008-07-03    16400     1920      804  ...     1680     4040      605   
2008-07-04    16350     1920      794  ...     1713     3956      615   
2008-07-07    15550   

185 A053350
150.50720929532523
77460554
186 A053450
133.25160568184845
77878174
187 A053470
95.63658099222953
78293799
188 A053620
122.88106955682942
78709209
189 A053980
146.04189942094385
79127499
190 A054040
193.7083527041686
79544379
191 A054090
171.19887145703933
79962303
192 A054170
376.9459836405443
80379663
193 A054210
186.04270424233178
80797977
194 A054920
246.1235540241201
81216177
195 A054930
22.616759018432656
81623177
196 A055250
110.10790574763267
82041177
197 A058220
663.0903991141112
82459530
198 A060370
280.4356848800296
82877290
199 A060380
77.05525632431016
83295400
200 A061040
283.28371146987416
83713391
201 A061050
39.84857541342897
84122891
202 A061140
38.92186435730271
84531391
203 A063160
94.38530156575776
84948093
204 A065060
201.35228192541106
85365771
205 A065940
350.427170721109
85783671
206 A066110
163.76126882231083
86200136
207 A066410
11.729039941720746
86592539
208 A066670
334.46046510072273
87010373
209 A066910
130.83490989399755
87426113
210 A067770


449.27985949598497
88939233
223 A086250
366.39900870311834
89340003
224 A086830
185.3149720692446
89740528
225 A088130
102.03634652337604
90141592
226 A088790
200.1031992667903
90542592
227 A088800
117.17491662672738
90943200
228 A089790
77.75327800967337
91340520
229 A090740
189.69594067608253
91740255
230 A092220
128.79836742533374
92138975
231 A092300
87.90686120287349
92536043
232 A095500
51.43678391409161
92933843
233 A950010
165.7879812107085
93333143
2009
            A000220  A000440  A000520  A000590  A000760  A000890  A000950  \
Code                                                                        
2010-07-01     6530     2965     5270    81600     8700     1694    40000   
2010-07-02     6410     2965     5299    78200     8450     1685    38650   
2010-07-05     6310     2970     5260    74300     8400     1685    38150   
2010-07-06     6330     2935     5327    73900     8590     1665    37550   
2010-07-07     6450     2915     5433    70900     8210     1670    374

20 A002460
160.73254827418347
11435976
21 A002530
6.0858428025253755
12003282
22 A002600
10.007348222983945
12578282
23 A002720
338.8825222741913
13152206
24 A002820
93.39758526563493
13725179
25 A002870
653.8892304790645
14299819
26 A003080
249.10065923011982
14875009
27 A003160
354.1061678902011
15450259
28 A003310
1424.3131753009327
16025555
29 A004100
739.617638588145
16600497
30 A004140
227.08071145287167
17175715
31 A004270
728.382940280477
17750835
32 A004320
108.77552416286896
18322155
33 A004450
29.283588947662942
18892005
34 A004590
548.0214503062637
19467405
35 A004740
566.3607508086386
20042461
36 A004770
1130.4961155630192
20617631
37 A005110
1567.908781530182
21192720
38 A005190
166.30708752068693
21767080
39 A005320
792.5930066412903
22342072
40 A005360
446.7566170974975
22916520
41 A005450
124.95603101445751
23487540
42 A005670
368.3882988614448
24062356
43 A005710
212.56834976785254
24636240
44 A005750
317.0372026565161
25211595
45 A005820
59.75311763463934
25779765
46

143.0901312472427
58046319
79 A008370
197.1548937879232
58784084
80 A008420
429.2703937417281
59521964
81 A008500
50.9203501541912
60246964
82 A008700
1001.8250708762176
60984701
83 A008720
39.27367432105172
61717901
84 A008830
398.0296912322223
62456191
85 A008900
139.17909090212487
63193586
86 A009070
473.2981264331874
63931466
87 A009140
58.13740765636003
64668066
88 A009300
114.29490359686879
65404506
89 A009440
360.16833035891335
66142506
90 A009770
32.77455065854814
66863402
91 A009810
997.763617886179
67601182
92 A010170
338.8458362715798
68337684
93 A010600
450.2104129486417
69075684
94 A010770
260.43918068281215
69812784
95 A010960
333.3386353208905
70550379
96 A011080
692.6314045363719
71288051
97 A011090
802.5489969954048
72025891
98 A011300
683.6528492923818
72763531
99 A011390
42.07094457183888
73500631
100 A011560
237.02891725064924
74238886
101 A012600
199.55272357723578
74975186
102 A012620
131.612313232758
75710096
103 A012790
346.80369996983205
76446730
104 A013120
63

...

87212038
84 A010660
333.1702813959204
88254328
85 A010770
333.7033538461539
89294953
86 A010960
342.47060123784263
90336343
87 A011300
1687.4158264874286
91378909
88 A011390
26.267581379577603
92411109
89 A011560
187.89603257103258
93448959
90 A012600
343.03387525303646
94491679
91 A012620
128.426475464191
95531039
92 A013360
1668.5167692307693
96573539
93 A013580
80.21715236686391
97613539
94 A013700
1779.5614006300868
98656033
95 A014160
1335.2406924061854
99698668
96 A014200
1158.692200854701
100740868
97 A014580
398.0240384615385
101783628
98 A014990
1247.3959100110417
102826120
99 A015260
1563.4527447814555
103868641
100 A017000
390.5704047824835
104909941
101 A017180
550.012120658877
105952741
102 A017370
347.60766025641027
106993741
103 A017480
258.44435706796304
108034771
104 A017650
1213.9964851795469
109076738
105 A018470
1022.3754713423832
110119178
106 A019010
1872.2136099986192
111161882
107 A019590
1826.3099488077598
112204528
108 A021050
789.4193647004018
113246797
109 A

278.10563430919365
214251882
131 A033290
598.840599424906
215890402
132 A033340
810.2830826786383
217530652
133 A033600
785.0828911120777
219171302
134 A033830
2229.3794054677205
220811846
135 A036170
698.2226563507414
222452146
136 A036200
169.9982638234814
224083334
137 A036620
829.1173534230634
225723925
138 A037230
1433.033399497155
227364710
139 A037330
841.4478166278167
229004660
140 A037350
390.67220057720056
230642660
141 A037400
378.94301210721534
232279400
142 A037460
370.38899377522404
233918500
143 A038010
475.6009398331137
235557250
144 A038110
644.7242602845746
237196230
145 A038620
1248.7239287855725
238836102
146 A039020
364.6273872053872
240474102
147 A039240
1095.3426184407492
242114412
148 A039740
371.22697792403676
243754232
149 A041190
3198.4858526788353
245394806
150 A043260
1676.0196551830873
247035610
151 A043360
472.8597240415684
248673450
152 A044780
973.7823397176513
250312955
153 A045300
661.8891659637928
251951574
154 A046310
669.7237724180582
253590624
155

129288491
68 A010040
664.7241949687943
131194171
69 A010100
655.5870926324535
133100221
70 A010170
1177.628666395333
135006961
71 A010240
830.1820885815664
136914301
72 A010660
791.6010122657426
138820611
73 A010770
529.9328998778999
140725011
74 A010960
549.7862938214523
142630041
75 A011300
1932.8859570014586
144536925
76 A011390
35.995442255857355
146391925
77 A011500
481.14966949821934
148299090
78 A011560
320.09369791282546
150206290
79 A012030
2399.69615039049
152113495
80 A012620
179.13224784605066
154019845
81 A012800
1680.8444401413565
155926645
82 A012860
732.0638678282577
157834237
83 A013360
1799.7721127928676
159741177
84 A013580
168.8281804920743
161639577
85 A013870
350.69088962508084
163543577
86 A014580
688.7214583250685
165449337
87 A014710
293.95353460099227
167350907
88 A015230
67.87485108906819
169234076
89 A016590
92.60963298837086
171129276
90 A016920
809.4011198813914
173036089
91 A017180
254.33388075729098
174941343
92 A017370
538.9148134351525
176845863
93 A01

364656114
165 A053700
359.00055989391484
366871144
166 A054090
884.2448920341136
369085564
167 A054210
405.6837828837829
371296864
168 A054340
92.87352010672765
373491064
169 A054930
142.90538416422288
375692064
170 A054940
1178.2092843326886
377906704
171 A056360
354.4053527272727
380119204
172 A058730
291.835764762247
382327894
173 A058850
873.7804554419939
384540949
174 A060380
617.8614935970585
386752894
175 A060540
1227.1653487786452
388967629
176 A061040
381.2450007823502
391181239
177 A062860
365.5170717071707
393393139
178 A063160
103.48689284925503
395597751
179 A064820
716.8393056781406
397810191
180 A065130
305.52185579937304
400021441
181 A065350
623.0755146400716
402236206
182 A065570
230.97324864916106
404441906
183 A065690
570.1501813501814
406656356
184 A065950
709.9466200466201
408868436
185 A066670
497.7603268641471
411080086
186 A066980
1960.2065969428802
413294886
187 A067310
442.1224460170568
415509306
188 A067570
625.7156651258347
417721806
189 A067770
654.3673425

451963364
187 A054210
861.4506484511214
454382774
188 A054540
631.2063421506261
456802659
189 A054620
494.0155759485002
459223259
190 A054930
147.60221476510068
461634059
191 A054940
1536.937347395334
464053259
192 A056700
1123.2836761706037
466473324
193 A058530
1613.7842147651008
468892824
194 A058850
1097.812390996667
471311709
195 A058860
843.4412272291468
473731119
196 A060380
708.8364047284483
476148939
197 A060540
1158.218335955814
478569159
198 A061040
582.5935793375814
480987369
199 A062860
543.9722072241913
483403719
200 A063160
133.28247561654285
485819265
201 A064820
881.8492976858474
488237610
202 A065130
386.072778651938
490657830
203 A065690
579.802711891653
493075155
204 A065950
672.4100894854587
495494355
205 A066130
536.735326418548
497911715
206 A066670
635.3481160492522
500331065
207 A066900
824.7619496244126
502749505
208 A067310
494.0155759485002
505170105
209 A067570
840.5126118568234
507589305
210 A067770
775.8577955601446
510007305
211 A067900
572.9411413367221

In [72]:
low_HML = get_onefactor_return(company_dict, price_df, 'HML_grp', 2008, 2019, 1, 100000000)
high_HML = get_onefactor_return(company_dict, price_df, 'HML_grp', 2008, 2019, 3, 100000000)

            A000050  A000070  A000270  A000320  A000360  A000390  A000430  \
Code                                                                        
2008-07-01    10415    52803    11550     7270  27039.0     3110     1766   
2008-07-02    10073    50955    11700     7100  25971.0     3200     1674   
2008-07-03    10301    53531    11400     7160  25771.0     3200     1626   
2008-07-04    10187    53139    11850     7000  25904.0     3080     1645   
2008-07-07    10301    54035    11950     6940  25303.0     3170     1689   

            A000440  A000480  A000490  ...  A086250  A086830  A086980  \
Code                                   ...                              
2008-07-01     4435    93500     3720  ...     1720   1732.0     1170   
2008-07-02     4395    92000     3720  ...     1590   1647.0     1140   
2008-07-03     4450    93000     3690  ...     1550   1565.0     1140   
2008-07-04     4105    92400     3720  ...     1480   1537.0     1140   
2008-07-07     4160   

113.78894426617511
56974935
297 A026180
199.21310822252104
57180900
298 A026250
82.57331478186195
57385654
299 A026870
6.028817748839453
57590854
300 A026890
132.59521994232108
57796114
301 A027050
166.27868307283006
58001954
302 A027390
15.273004963726613
58204454
303 A027710
248.41634579555335
58410294
304 A027740
288.371422392041
58616214
305 A027830
301.00082775227634
58822399
306 A027840
84.15737428992216
59028199
307 A028080
11.654828274846492
59222800
308 A028090
126.49421288976029
59428180
309 A029530
2.832219327064688
59573780
310 A030190
307.28102386037153
59779777
311 A030270
14.604445885416439
59977429
312 A030420
39.65107057890563
60180229
313 A031310
65.45573555882835
60384979
314 A031330
48.44585691031703
60589267
315 A031990
0.8815116161193215
60589267
316 A032190
76.7916450690165
60793327
317 A032380
118.15791805748383
60999237
318 A032750
126.49421288976029
61204617
319 A032930
98.13687149467363
61410515
320 A033160
55.27763190624914
61615665
321 A033240
27.4914089347

7968884
46 A001940
14.685758950742699
8148644
47 A002000
9.334908164729518
8330444
48 A002030
3.9912190692673564
8472179
49 A002070
28.31308482395439
8658659
50 A002100
44.21222624326759
8846319
51 A002140
378.6448693324021
9034563
52 A002170
7.129117010492864
9219713
53 A002200
14.789423131571471
9398213
54 A002210
129.59803775088403
9585908
55 A002220
152.68432787654757
9773628
56 A002230
72.52505574136009
9960828
57 A002240
6.498660908724023
10134924
58 A002250
13.66414093677799
10314324
59 A002290
137.63879191790969
10502014
60 A002300
4.785917383947621
10659614
61 A002310
14.789423131571471
10838114
62 A002450
188.56514492753627
11026114
63 A002460
31.16779255000599
11213664
64 A002530
0.882733632598536
11213664
65 A002600
5.021708253729328
11401414
66 A002620
19.407692973192287
11586018
67 A002710
46.34188865262626
11773192
68 A002720
93.44159808103878
11960866
69 A002810
57.29721814875
12148453
70 A002820
26.62597358479755
12332585
71 A002880
419.9669152060941
12520716
72 A00290

60134137
338 A035890
92.25300632462634
60322185
339 A036010
64.35670475342535
60509705
340 A036200
247.46082011487698
60697919
341 A036530
18.180210656337856
60884615
342 A036540
70.3076603010948
61072355
343 A036580
124.87757942220944
61259595
344 A037070
103.09740017907941
61447982
345 A037230
327.93938248267176
61636007
346 A037240
36.973557828928676
61819607
347 A037350
62.02800819984745
62008087
348 A037460
68.56914361001319
62195087
349 A038110
92.6610048783962
62382307
350 A038340
31.29711949004751
62569082
351 A038460
49.1055064915459
62757242
352 A038500
49.067172762824946
62945549
353 A038680
183.07295624032645
63134039
354 A038950
85.32359498983541
63321889
355 A039740
69.83894256575417
63508189
356 A039830
69.83894256575417
63694489
357 A040610
206.08212560386477
63882979
358 A041190
502.84038647343
64071229
359 A041550
132.79235558277202
64258669
360 A041650
123.64927536231886
64446244
361 A042420
10.248105702583493
64630244
362 A042870
10.192710536623581
64815244
363 A043

23409299
112 A004150
16.077415945627433
23639267
113 A004200
15.7411920563013
23859467
114 A004250
125.5873366230995
24089467
115 A004270
284.5821421016048
24320075
116 A004320
37.39169892985487
24548735
117 A004360
14.625360720664752
24769935
118 A004450
11.554034969325155
24989935
119 A004550
86.54707842191127
25219555
120 A004560
25.44941623199373
25446555
121 A004590
178.44069450695218
25677065
122 A004690
2.254445847673201
25882065
123 A004700
23.579663202704396
26107465
124 A004740
240.4585841690979
26338105
125 A004890
3.329693074733474
26546305
126 A004960
24.324284145947694
26774305
127 A004970
18.195330660354575
27002905
128 A004980
52.578088597611625
27231445
129 A004990
6.5102324098183715
27444415
130 A005010
14.765539896901155
27663515
131 A005030
138.45458321540028
27893837
132 A005110
585.0144288265901
28124912
133 A005160
44.35330122581633
28354152
134 A005190
78.86713289641743
28582692
135 A005300
2.9422414263805634
28739770
136 A005360
136.49184842675905
28970018
137 

...

147 A035620
109.84256353337416
35583744
148 A035900
28.102101905090166
35835464
149 A036120
212.30075304769795
36087744
150 A036180
52.197912422884414
36339424
151 A036480
22.317835346886977
36588464
152 A036490
1.337416072666811
36777364
153 A036570
0.6846555450589717
36777364
154 A036620
206.91064383846074
37028890
155 A036670
32.306636333345345
37279130
156 A036800
9.462093487893654
37519430
157 A036930
19.814736951118476
37761680
158 A037230
106.82363472590299
38012370
159 A037950
112.53358402082877
38263810
160 A038070
24.252461949386635
38513818
161 A038290
9.006698614144762
38766268
162 A038340
70.17719336854461
39018268
163 A038530
72.63884304967239
39268684
164 A038880
312.67066352321854
39520780
165 A039030
2.9376499549623323
39692780
166 A039130
2.7884977497434944
39873980
167 A039200
34.419331897378825
40123540
168 A039230
118.60934090458242
40374880
169 A039290
26.343889064312886
40624220
170 A039420
84.77781749220154
40874540
171 A039440
17.131477326016178
41125239
172 A0

397 A093640
69.12117541087841
96344943
398 A094170
50.92479260769211
96592993
399 A094190
176.05428301516417
96845553
400 A094280
15.839366528323547
97084803
401 A094360
25.518979406743494
97332303
402 A094480
51.317874492537186
97583376
403 A094850
23.28459872136042
97832926
404 A094860
310.74771971311264
98084956
405 A094940
39.66058023967984
98333386
406 A095270
11.354512185472386
98578136
407 A095610
9.305263209088787
98822486
408 A095700
5.444782244111219
99054486
409 A096530
6.800481726157755
99277386
410 A096690
39.29049706481502
99528156
411 A097520
18.531350115657638
99773550
412 A097800
126.0039382178357
100026180
413 A098460
4.210631602112676
100266180
414 A099190
9.27111545419305
100511430
415 A099320
11.24184114834515
100758633
416 A099520
172.802938527196
101010097
417 A100120
4.652631604544394
101227297
418 A100700
63.3177684528222
101478667
419 A101140
88.64487583395108
101729467
420 A101400
259.91553099460964
101981215
421 A101490
41.552285547164566
102230495
422 A1024

78 A018120
8.837433552054968
20170087
79 A018260
1.378594659904025
20366587
80 A018290
40.013862728381234
20637387
81 A018620
74.74995879446494
20905563
82 A018680
23.454013045120426
21171213
83 A018880
25.79941434963247
21433713
84 A019590
233.52918161305254
21703993
85 A019660
55.73947544673682
21971293
86 A020120
92.93099508443943
22239473
87 A020150
6.426900371794566
22492373
88 A020760
67.89319565692756
22759703
89 A021080
98.86636885808063
23028223
90 A021240
3.213450185897283
23281123
91 A022100
41.04452282896075
23551723
92 A023450
12.414933577962463
23813563
93 A023770
39.20316218106237
24083053
94 A024890
164.178091315843
24353653
95 A025320
74.11596461590723
24624123
96 A025440
247.39164444853054
24894588
97 A025560
1242.6323425281694
25165344
98 A025620
12.718021158269528
25420944
99 A025770
29.15972558354585
25690354
100 A026180
118.29425793499604
25960574
101 A026960
10.561163768855398
26217074
102 A027040
271.98177778227
26486990
103 A027050
48.201752788459245
26756750
1

328 A078130
214.9951195802706
85612985
329 A078140
30.748450700470027
85877285
330 A078160
6.322647932574185
86134355
331 A078340
1.6538086121559277
86298155
332 A078590
342.9036084444822
86568335
333 A078600
11.358232732542596
86830685
334 A078650
194.18914026605086
87101315
335 A078940
104.18994256582344
87371715
336 A079190
86.27192696533152
87641755
337 A079950
82.0890456579215
87912355
338 A079970
22.204413989437782
88180755
339 A080160
10.62328826161337
88435755
340 A080220
59.14712896749802
88705975
341 A080440
60.332706162837624
88975375
342 A081000
10.877960513638556
89224405
343 A082270
22.204413989437782
89492805
344 A082800
150.91579424576096
89762055
345 A082920
22.025681004239445
90032633
346 A083450
36.90651916500558
90296873
347 A083640
74.42138754701675
90566233
348 A083660
24.515280603723163
90831433
349 A083790
15.905927465864655
91086898
350 A084180
46.545335166862706
91354618
351 A084650
32.403570654442696
91622138
352 A084990
1.5402372705576646
91798016
353 A08566

578 A234080
10.835754026845638
150060210
579 A237880
9.606164917416345
150314010
580 A238090
137.43980247140587
150584037
581 A238120
19.14444174354353
150852887
582 A239610
8.781000021754974
151099687
583 A240810
10.033105580412627
151369687
584 A241520
66.558685668585
151638307
585 A241590
24.739164444853053
151901107
586 A241690
16.93086566694631
152157107
587 A241710
8.086383602123611
152425107
588 A241790
28.818494752249038
152688307
589 A243070
3.7717916858737826
152903770
590 A246690
104.18994256582344
153174170
591 A250060
130.2374282072793
153444570
592 A252500
51.89537369178945
153710790
593 A900100
82.84215616854463
153978930
594 A950130
59.734035429138025
154246495
595 A950140
53.53633412473141
154514675
2018
                펀드포트폴리오    현금포트폴리오      종합포트폴리오      일변화율      총변화율
Code                                                               
2008-07-01   93533000.0  6467000.0  100000000.0       NaN  0.000000
2008-07-02   89754289.0  6467000.0   96221289.0 -0.037787 -0.0377

In [73]:
low_SMB = get_onefactor_return(company_dict, price_df, 'SMB_grp', 2008, 2019, 1, 1000000000)
high_SMB = get_onefactor_return(company_dict, price_df, 'SMB_grp', 2008, 2019, 2, 1000000000)

            A000250  A000420  A000440  A000590  A000760  A000890  A001000  \
Code                                                                        
2008-07-01     3045    865.0     4435    81635     5290     1762     1102   
2008-07-02     2755    861.0     4395    78801     5290     1665     1266   
2008-07-03     2790    823.0     4450    82089     5210     1665     1180   
2008-07-04     2725    834.0     4105    83109     5120     1627     1060   
2008-07-07     2660    796.0     4160    83336     5300     1617     1146   

            A001070  A001140  A001360  ...  A088800   A088810  A088910  \
Code                                   ...                               
2008-07-01     7720      742     2905  ...     3949  107927.0     4490   
2008-07-02     7670      738     2766  ...     3834   98333.0     4325   
2008-07-03     7500      734     2546  ...     3805   97134.0     4970   
2008-07-04     7600      740     2635  ...     3782   99532.0     5100   
2008-07-07     7

1151.5775460803757
326740849
230 A024940
1312.9045387109904
328164369
231 A024950
534.1212690294055
329588547
232 A025270
1202.110906752257
331012917
233 A025340
109.57703265395574
332429917
234 A025440
1202.110906752257
333854287
235 A025550
244.54960077277676
335275587
236 A025620
16.049454403612387
336695699
237 A025820
982.4147755182239
338119599
238 A025870
415.9128246719488
339540974
239 A025880
786.1486890184462
340965206
240 A025890
284.331621656971
342388046
241 A025920
823.4112280355056
343811836
242 A025970
879.3218669761881
345235816
243 A026040
913.1419387829645
346660096
244 A026260
1.2081885539845796
347839135
245 A026540
4451.566951566952
349263455
246 A026910
602.3261837215326
350687185
247 A027040
2335.2482368875812
352111535
248 A027050
1148.7914713721166
353535055
249 A027710
1716.2667765077406
354959335
250 A027740
1992.3096846173771
356383615
251 A027830
2079.564123359744
357807730
252 A028040
73.60623285802845
359220499
253 A028090
873.9272542953526
360643489
254

479 A052210
89.84556445925101
680569373
480 A052270
312.39066326785627
681992093
481 A052290
62.574189523453754
683403523
482 A052350
102.48211687060609
684821323
483 A052400
347.1853337805081
686245064
484 A052460
696.9185051376833
687667688
485 A052510
543.0809853226933
689091977
486 A052560
1121.6546649617517
690515647
487 A052600
582.618169530235
691938637
488 A052650
1312.9045387109904
693362157
489 A052670
778.8416755065198
694785119
490 A052810
919.0331770976933
696209569
491 A052880
350.8624198279371
697630569
492 A052900
547.4640370874038
699053863
493 A053050
679.9529472560499
700476368
494 A053060
2088.7117661311213
701900384
495 A053160
386.04374647735085
703324724
496 A053270
1077.535116869459
704748518
497 A053290
485.34971873983807
706171993
498 A053330
183.09786947319083
707595733
499 A053350
512.4105843530305
709019093
500 A053440
107.91677458344127
710431493
501 A053450
453.6628740450397
711853913
502 A053470
325.6003256003256
713275788
503 A053610
660.102606349131
71

26 A002760
4303.800753530615
33617836
27 A002870
1014.7172508324214
34865056
28 A002880
2779.7376804540722
36112827
29 A003060
696.8744938715122
37359363
30 A003160
1145.0479069026408
38607413
31 A003280
743.3604636830723
39854910
32 A003310
2600.21295525808
41102910
33 A003580
249.62044370477568
42347910
34 A003680
97.507985822178
43589510
35 A003720
1205.895863308095
44836685
36 A003780
1109.4241942434473
46084310
37 A004060
5032.670235983381
47332246
38 A004100
2228.753961649783
48579926
39 A004250
645.014066420609
49828001
40 A004270
1518.372528617857
51075797
41 A004540
238.1874462831829
52322917
42 A004590
971.2857731703334
53570652
43 A004620
126.19840429968437
54816792
44 A004740
948.4059411275672
56064360
45 A004770
968.2716978462981
57312112
46 A004780
461.4056260716741
58559117
47 A004830
646.6850873180717
59805897
48 A004840
745.1356528500767
61053772
49 A004870
147.5472536380043
62297245
50 A004910
366.0123807987913
63545305
51 A004920
2003.3743475503666
64793174
52 A00503

276 A036000
260.021295525808
344781903
277 A036010
425.9734534211189
346027153
278 A036090
1066.7540329263918
347274373
279 A036120
2684.0907925244696
348522433
280 A036170
284.95484441184436
349766353
281 A036180
391.86882842193984
351011688
282 A036200
1637.929420635011
352259082
283 A036260
140.5520516355719
353502282
284 A036480
269.859939140298
354746407
285 A036620
696.4856130155571
355993639
286 A036670
260.021295525808
357241639
287 A036690
1149.2653945892066
358489453
288 A036800
405.22799302723325
359736853
289 A036810
1040.085182103232
360984853
290 A036850
405.8869003329686
362230228
291 A036920
498.2443986123267
363477718
292 A037020
215.19003767653075
364724718
293 A037070
682.3959642011364
365972096
294 A037230
2170.612553954571
367219846
295 A037240
244.72592520076046
368464246
296 A037270
1217.6607009989057
369711671
297 A037320
541.9462520728955
370957594
298 A037330
1010.6090838250027
372204944
299 A037350
410.55994030390735
373451344
300 A037380
182.63128746325407
3

...

1324225147
785 A141080
48.25440605533826
1325924347
786 A143240
84.774490042629
1327616947
787 A143540
265.57928705829823
1329321427
788 A145020
2.9707929988851727
1330471427
789 A145990
16.829615510925855
1332095427
790 A151860
397.71966806960984
1333800542
791 A152330
88.05185434840074
1335507742
792 A155660
261.994781343401
1337209462
793 A159580
252.3199371283566
1338915502
794 A160550
206.05620921097398
1340623242
795 A161000
108.45752218152218
1342324242
796 A161390
26.36120330800886
1344009042
797 A161890
23.27256095856913
1345697242
798 A163560
182.13092806898115
1347404220
799 A166480
181.91756915431037
1349103810
800 A168330
144.76321816601478
1350803010
801 A170900
18.668917752557096
1352450010
802 A171120
139.15004678714357
1354156374
803 A178920
79.63664216125754
1355850924
804 A180640
67.51802270193573
1357546024
805 A181710
20.730655028628327
1359194024
806 A182400
101.83046046849324
1360888299
807 A183190
20.56246207428286
1362549779
808 A185750
16.133414944833532
13642

83016210
51 A001800
62.34311564985056
84656110
52 A001820
18.342329354155144
86274310
53 A001880
134.06304137711768
87922510
54 A001940
132.9818878176248
89559310
55 A002020
40.91750394388455
91171310
56 A002030
13.741461741154561
92731310
57 A002100
166.5631726200553
94374710
58 A002150
253.6885244520842
96019210
59 A002170
42.22728319945064
97659310
60 A002240
58.06251439924463
99306510
61 A002270
1.895374033262698
100176510
62 A002310
50.42738253634701
101811510
63 A002320
82.24316254057592
103455610
64 A002350
163.2648919741136
105101910
65 A002360
1293.3140462263116
106750485
66 A002380
5.169201908898268
108345485
67 A002390
62.57971191417637
109979185
68 A002450
721.6522577411586
111626670
69 A002460
116.94861056301754
113262270
70 A002600
7.512416441633473
114798770
71 A002620
59.962742143219906
116421270
72 A002780
863.33791043903
118069600
73 A002790
13.773139963069621
119626012
74 A002810
113.30828069391517
121270501
75 A002840
43.31543799255424
122907468
76 A002960
4.7248579

53.79491106705861
487493655
301 A018260
8.391732361010419
489065655
302 A018310
119.49097166221358
490707855
303 A018670
18.43913996666086
492317559
304 A018880
157.0452770417664
493966059
305 A019170
244.65510518376072
495610619
306 A019210
119.92548428643981
497246869
307 A019440
100.85476507269402
498881869
308 A019550
1820.061157768816
500530789
309 A019680
213.59785089877556
502175149
310 A020000
39.97516142881327
503783899
311 A020150
39.121599263073485
505427749
312 A020560
409.6833314133037
507073974
313 A020710
237.2626487681363
508721124
314 A020760
413.2770448467537
510368994
315 A021240
19.560799631536742
511970694
316 A021320
189.5374033262698
513614994
317 A022100
249.84475893008295
515258394
318 A022220
597.4548583110679
516906114
319 A023000
542.4261213613643
518553794
320 A023160
153.39306129660906
520198544
321 A023410
245.38324537776003
521844944
322 A023530
8.04378248262706
523484944
323 A023590
72.96351366984723
525112144
324 A023890
34.31790653358059
526745844
325

550 A065060
830.7180901453639
896063723
551 A065350
417.46212884520185
897710873
552 A065510
126.8442622260421
899348873
553 A065620
126.8442622260421
900986873
554 A065650
291.8540546793889
902631023
555 A065660
15.197930036299976
904258523
556 A065680
164.89754089385474
905898523
557 A066570
20.689779284047017
907492523
558 A066620
87.01717197564894
909141173
559 A066700
162.46063142251697
910785473
560 A066970
33.48173419164563
912410723
561 A066980
1505.913615468993
914058698
562 A067000
121.69560213568616
915698248
563 A067080
78.5226385208832
917336248
564 A067160
35.96456726147323
918940998
565 A067170
140.9380691400468
920578998
566 A067280
47.45252975362726
922212248
567 A067290
225.88704232034897
923854748
568 A067390
115.90464672373285
925490853
569 A067630
94.52966114071012
927130589
570 A067830
386.176910758442
928778809
571 A067990
308.7968930596531
930423529
572 A068240
126.35826888417988
932067829
573 A068400
163.2648919741136
933714129
574 A068760
19.62411825746831
935

800 A178920
32.782811310905515
1304541225
801 A180400
168.7794686733416
1306182585
802 A180640
102.73990086844532
1307819685
803 A181710
25.68497521711133
1309424685
804 A182400
107.7761705188593
1311061785
805 A183190
12.68442622260421
1312621785
806 A185490
75.81496133050793
1314253035
807 A185750
18.500167266203846
1315857429
808 A187420
191.51863053873953
1317501939
809 A191420
39.63883194563816
1319124339
810 A192400
9.035481692813958
1320766839
811 A192410
267.2569544470903
1322414229
812 A192440
37.51934946390324
1324040379
813 A192820
10.536584082674423
1325605379
814 A193250
651.7689363393467
1327252409
815 A194370
128.82620382332402
1328890809
816 A194480
118.63132438406815
1330531009
817 A194510
155.56371782439126
1332174009
818 A195870
83.70433547911408
1333809109
819 A196170
53.97628179831579
1335428259
820 A196300
103.06096305865921
1337076259
821 A200130
61.99155672701306
1338698859
822 A200230
192.8626209284851
1340340459
823 A200670
61.715461242507104
1341970318
824 A2

In [74]:
HML_return = low_HML.iloc[1:]['일변화율']-hihg_HML.iloc[1:]['일변화율']
HML_return_df = pd.DataFrame(HML_return)

In [75]:
SMB_return = low_SMB.iloc[1:]['일변화율']-high_SMB.iloc[1:]['일변화율']
SMB_return_df = pd.DataFrame(SMB_return)

In [76]:
factor_df = pd.concat([HML_return_df, SMB_return_df], axis=1)
factor_df.columns=['SMB_rtn', 'HML_rtn']
factor_df.index.name ='Date'
factor_df.to_excel('two_factor_rtn.xlsx')

In [65]:
factor_df.head()

,SMB_rtn,HML_rtn
Date,,
2008-07-02,0.009643,0.011436
2008-07-03,0.002402,-0.007161
2008-07-04,0.000008,0.009160
2008-07-07,0.002170,0.005199
2008-07-08,0.007185,0.004691


In [68]:
target_com_df = company_dict[2019]

KeyError: 2019

In [77]:
group_df

,Name,SMB_grp,HML_grp
Code,,,
A000020,동화약품,2,2
A000040,KR모터스,1,2
A000050,경방,2,1
A000070,삼양홀딩스,2,1
A000080,하이트진로,2,2
A000100,유한양행,2,2
A000120,CJ대한통운,2,2
A000140,하이트진로홀딩스,2,1
A000150,두산,2,1


In [53]:
S1B1_df = target_com_df[(target_com_df['SMB_grp'] == 1) & (target_com_df['HML_grp'] == 1)]

In [54]:
company_codes = S1B1_df.index.values.tolist()

In [55]:
test = price_df.loc['2015-01':'2015-07'][company_codes]
test = test.dropna(axis=1)
test

,A000220,A000440,A000520,A000590,A000760,A000910,A000950,A001070,A001140,A001380,...,A115440,A119830,A121800,A121890,A122690,A130740,A900070,A900090,A900110,A950010
Code,,,,,,,,,,,,,,,,,,,,,
2015-01-02,7630,4620,5270,81300,13400,3895,34100,6100,1057,1310,...,3850.0,2530.0,2227.0,2385.0,3775.0,2730.0,833.0,2217.0,1590.0,690.0
2015-01-05,7660,4600,5308,81400,13400,3875,34500,6300,1057,1310,...,3820.0,2480.0,2241.0,2399.0,3800.0,2755.0,821.0,2091.0,1478.0,690.0
2015-01-06,7630,4590,5346,80000,13450,3845,34750,6360,1053,1310,...,3815.0,2460.0,2264.0,2355.0,3760.0,2730.0,848.0,2156.0,1256.0,690.0
2015-01-07,7590,4540,5356,80000,13550,3930,34750,6400,1136,1335,...,3780.0,2455.0,2273.0,2388.0,3725.0,2740.0,825.0,2132.0,1223.0,690.0
2015-01-08,7570,4530,5356,80000,13600,3925,35150,6500,1165,1330,...,3785.0,2520.0,2295.0,2441.0,3890.0,2745.0,829.0,2066.0,1403.0,690.0
2015-01-09,7620,4570,5346,81100,13450,3995,34900,6730,1140,1305,...,3755.0,2510.0,2273.0,2509.0,3940.0,2755.0,852.0,2037.0,1489.0,690.0
2015-01-12,8080,4510,5318,81600,13450,4065,35200,6820,1136,1315,...,3780.0,2530.0,2295.0,2511.0,3860.0,2755.0,871.0,2062.0,1418.0,690.0
2015-01-13,7860,4690,5279,79800,13350,4070,35400,6960,1111,1320,...,3790.0,2520.0,2273.0,2500.0,3890.0,2705.0,905.0,2275.0,1496.0,690.0
2015-01-14,7990,4730,5337,80800,13300,4080,36050,7300,1082,1330,...,3805.0,2520.0,2318.0,2526.0,3850.0,2710.0,840.0,2095.0,1376.0,690.0


In [56]:
rtn = (test.iloc[-1]/test.iloc[0]) -1

In [59]:
rtn.sort_values()

A900110   -0.393711
A048870   -0.375000
A077280   -0.341040
A009270   -0.214612
A002710   -0.195066
A005430   -0.174233
A096350   -0.153499
A900090   -0.143888
A038680   -0.141531
A073570   -0.111279
A101930   -0.096552
A049120   -0.095890
A038010   -0.094828
A006660   -0.086957
A121800   -0.085766
A122690   -0.059603
A039240   -0.058978
A010100   -0.051051
A031510   -0.047619
A027740   -0.036697
A057880   -0.033708
A000590   -0.029520
A005450   -0.023747
A045060   -0.020210
A010660   -0.015974
A023600   -0.011645
A049800   -0.007282
A007280   -0.005566
A058730   -0.003562
A062860   -0.001460
             ...   
A005320    1.194030
A007770    1.244224
A013700    1.297090
A065950    1.326531
A001810    1.359743
A037400    1.385757
A031860    1.395760
A025270    1.445323
A002720    1.452812
A053810    1.488119
A098660    1.504228
A046310    1.516563
A006090    1.582217
A078130    1.653846
A080010    2.049645
A004590    2.126168
A900070    2.127251
A002820    2.196643
A033290    2.586022


In [31]:
df.columns.unique()

Index(['Name', '결산월', '상장일', '당기순이익', 'ROA', 'ROA.1', '영업활동으로인한현금흐름',
       '영업활동으로인한현금흐름.1', '비유동부채', '비유동부채.1', '자산총계', '자산총계.1', '유동비율',
       '유동비율.1', '매출총이익률', '매출총이익률.1', '총자산회전율', '총자산회전율.1', '수정주가', '수정주가.1',
       '수정DPS(현금)', '12개월전대비수익률(현금배당포함)', 'P/E(Adj., FY End)',
       'P/B(Adj., FY End)', 'P/S(Adj., FY End)', 'EV/EBITDA',
       '시가총액규모별(1:KS대형주, 2:KS중형주, 3:KS소형주, 4:KQ대형주, 5:KQ중형주, 6:KQ소형주)', '시가총액',
       '보통주수정기말발행주식수', '보통주수정기말발행주식수.1'],
      dtype='object')